In [1]:
# !pip install rank-bm25 nltk
# nltk.download('stopwords')

In [2]:
from beir.datasets.data_loader import GenericDataLoader

corpus_name = 'scifact'
corpus_name = 'trec-covid'
# corpus_name = 'nfcorpus'

corpus, queries, qrels = GenericDataLoader(f'data/{corpus_name}').load(split="test")
corpus_text = [v['text'] for k,v in corpus.items()]

/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/beir/datasets/data_loader.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
100%|███████████████████████████████████████████████████████████████████████████████████| 171332/171332 [00:01<00:00, 116744.32it/s]


In [3]:
from utils.preprocessor import Stopwords_preprocessor
from utils.markdown import beir_metrics_to_markdown_table
from IPython.display import Markdown

from rank_bm25 import BM25Okapi as BM25
from transformers import logging, AutoTokenizer

from beir.retrieval.evaluation import EvaluateRetrieval

In [4]:
%time #build
preprocessor = Stopwords_preprocessor()
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')
# logging.get_logger().setLevel(logging.ERROR)

def simple_split(text):
    return text.split(' ')

bm25 = BM25([v['text'] for k,v in corpus.items()], simple_split)

CPU times: user 2 µs, sys: 4 µs, total: 6 µs
Wall time: 13.1 µs


In [5]:
%time #query
results = {qid: dict(zip(corpus.keys(), bm25.get_scores(bm25.tokenizer(query)))) \
                 for qid, query in queries.items()}

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 16.5 µs


In [6]:
%time #evaluate
metrics = EvaluateRetrieval.evaluate(qrels, results, [1, 3, 5, 10, 100, 1000])

flatten_metrics = {k: v for metric_type in metrics for k, v in metric_type.items()}
metric_names, metric_values = zip(*flatten_metrics.items())
print(*metric_names, sep='\t')
print(*metric_values, sep='\t')
print()

Markdown(beir_metrics_to_markdown_table(*metrics))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
NDCG@1	NDCG@3	NDCG@5	NDCG@10	NDCG@100	NDCG@1000	MAP@1	MAP@3	MAP@5	MAP@10	MAP@100	MAP@1000	Recall@1	Recall@3	Recall@5	Recall@10	Recall@100	Recall@1000	P@1	P@3	P@5	P@10	P@100	P@1000
0.47	0.43592	0.42873	0.40482	0.27558	0.23447	0.00119	0.00299	0.00468	0.00807	0.03569	0.07884	0.00119	0.00338	0.00566	0.01069	0.0635	0.22518	0.54	0.46667	0.464	0.44	0.2888	0.11302



||NDCG|MAP|Recall|P|
|-|-|-|-|-|
|@1|0.4700|0.0012|0.0012|0.5400|
|@3|0.4359|0.0030|0.0034|0.4667|
|@5|0.4287|0.0047|0.0057|0.4640|
|@10|0.4048|0.0081|0.0107|0.4400|
|@100|0.2756|0.0357|0.0635|0.2888|
|@1000|0.2345|0.0788|0.2252|0.1130|

In [7]:
flatten_metrics = {k: v for metric_type in metrics for k, v in metric_type.items()}
metric_names, metric_values = zip(*flatten_metrics.items())
print(*metric_names, sep='\t')
print(*metric_values, sep='\t')
print()

NDCG@1	NDCG@3	NDCG@5	NDCG@10	NDCG@100	NDCG@1000	MAP@1	MAP@3	MAP@5	MAP@10	MAP@100	MAP@1000	Recall@1	Recall@3	Recall@5	Recall@10	Recall@100	Recall@1000	P@1	P@3	P@5	P@10	P@100	P@1000
0.47	0.43592	0.42873	0.40482	0.27558	0.23447	0.00119	0.00299	0.00468	0.00807	0.03569	0.07884	0.00119	0.00338	0.00566	0.01069	0.0635	0.22518	0.54	0.46667	0.464	0.44	0.2888	0.11302

